# 37

In [1]:
# 관련 라이브러리 설치
# datasets==1.4.1 버전을 설치해야 함에 유의
!pip install datasets==1.4.1
!pip install transformers
!pip install soundfile
!pip install jiwer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.2
    Uninstalling huggingface-hub-0.0

#38

In [2]:
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_name = "elgeish/wav2vec2-base-timit-asr"

# 모델 및 프로세서 불러오기
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# 모델을 eval 모드로 전환
model.eval()


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

#39

In [3]:
# 런타임 1분 30초
dataset = load_dataset("timit_asr")

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/72abf9645fc2cfd87e21af1e381de0ab6ed1be4123c441ca97dc6766b96816a6. Subsequent calls will reuse this data.


#40

In [4]:
import soundfile as sf
import IPython.display as ipd
import numpy as np
import random

# dataset['test']["file"]에 있는 첫번째 데이터를 df.read( )에 투입
speech, _ = sf.read(dataset['test']["file"][0])
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)


#41

In [5]:
# 음성 데이터를 프로세서로 처리
inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding="longest")

# 음성 데이터 처리 결과중에 inputs['input_values'] 차원 확인
inputs['input_values'].shape


torch.Size([1, 44749])

#42

In [6]:
# 그래디언트(기울기) 계산 방지, 이는 모델 eval 모드에 필요한 조치임
with torch.no_grad():

  # 모델에 input.input_values 투입
  outputs = model(inputs.input_values)

  # 로짓(outputs.logits)의 차원 확인
  print(outputs.logits.shape)

  # 로짓(output.logits)의 마지막 축 기준으로 argmax 적용
  # 가장 큰 값의 인덱스를 predicted_ids에 저장
  predicted_ids = outputs.logits.argmax(-1)

  # 위의 predicted_ids 출력
  print(predicted_ids)

  # predicted_ids의 첫번째 데이터를 processor.tokenizer.batch_decode)에 넣고 디코딩
  # predicted_ids[0]은 입력 문장 중 첫번째라는 뜻이나
  # 입력 문장을 하나만 넣었으므로 predicted_ids[0]은 입력 문장 그 자체를 의미함
  # 디코딩 결과를 출력
  print(f"predicted:{processor.tokenizer.batch_decode(predicted_ids)[0]}")

  # dataset['test']['text]에 있는 첫번째 문장, 즉 정답 문장 출력
  print(f"answer:{dataset['test']['text'][0]}")


torch.Size([1, 139, 32])
tensor([[ 0,  0,  0,  0,  0,  0,  6,  6, 11,  5,  5,  4, 24, 24,  0,  0,  0, 16,
         16,  9,  9, 21, 21,  0,  0, 15, 15, 15, 15,  0,  0, 16,  5,  5,  4,  4,
         18,  0,  7, 12, 12, 12,  4,  4,  0, 23, 23, 15, 15,  5,  5,  5, 12, 12,
         12,  0,  9,  9,  6,  6,  6, 15, 15, 15, 22, 22,  4,  4, 12, 12,  0,  0,
          0, 10, 10,  6,  6,  0,  0,  0, 16, 16, 16,  0,  0,  0,  0,  7,  7,  6,
          6,  0,  5,  5, 14, 14,  4,  4,  9,  9,  9,  5,  5,  7,  7, 13,  4,  4,
          6, 11,  5,  5,  4, 12, 12, 11, 11, 11,  0,  0,  0,  8,  8,  8,  8,  8,
         13, 13, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
predicted:the bunglue was plesntly situated near the shor
answer:The bungalow was pleasantly situated near the shore.
